In [42]:
import torch
from torch import nn
from torch._export import capture_pre_autograd_graph
from torch.fx.experimental.proxy_tensor import make_fx


class MyAct(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.act = MyReLU()

    def forward(self, x):
        return self.act(x)


class MyReLU(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.myrelu = torch.nn.ReLU()

    def forward(self, x):
        return self.myrelu(x)


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 3, 3, 3)
        self.relu = nn.ReLU()
        self.custom_act = MyAct()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = torch.sigmoid(x)
        x = self.custom_act(x)
        return x

from torch import _guards
from torch._subclasses import FakeTensorMode
x  = torch.randn(1, 1, 10, 10)
net = Net()
y = net(x)
# fake_mode = _guards.detect_fake_mode(x)
# fake_tensor = fake_mode.from_tensor(x)


with FakeTensorMode(allow_non_fake_inputs=True):
    exported = make_fx(net, tracing_mode="real")(x)
exported.print_readable()

class <lambda>(torch.nn.Module):
    def forward(self, arg0_1: f32[1, 1, 10, 10]):
        # No stacktrace found for following nodes
        _param_constant0 = self._param_constant0
        _param_constant1 = self._param_constant1
        convolution: f32[1, 3, 3, 3] = torch.ops.aten.convolution.default(arg0_1, _param_constant0, _param_constant1, [3, 3], [0, 0], [1, 1], False, [0, 0], 1);  arg0_1 = _param_constant0 = _param_constant1 = None
        relu: f32[1, 3, 3, 3] = torch.ops.aten.relu.default(convolution);  convolution = None
        detach: f32[1, 3, 3, 3] = torch.ops.aten.detach.default(relu)
        sigmoid: f32[1, 3, 3, 3] = torch.ops.aten.sigmoid.default(relu);  relu = None
        detach_1: f32[1, 3, 3, 3] = torch.ops.aten.detach.default(sigmoid)
        relu_1: f32[1, 3, 3, 3] = torch.ops.aten.relu.default(sigmoid);  sigmoid = None
        detach_2: f32[1, 3, 3, 3] = torch.ops.aten.detach.default(relu_1)
        return relu_1
        


'class <lambda>(torch.nn.Module):\n    def forward(self, arg0_1: f32[1, 1, 10, 10]):\n        # No stacktrace found for following nodes\n        _param_constant0 = self._param_constant0\n        _param_constant1 = self._param_constant1\n        convolution: f32[1, 3, 3, 3] = torch.ops.aten.convolution.default(arg0_1, _param_constant0, _param_constant1, [3, 3], [0, 0], [1, 1], False, [0, 0], 1);  arg0_1 = _param_constant0 = _param_constant1 = None\n        relu: f32[1, 3, 3, 3] = torch.ops.aten.relu.default(convolution);  convolution = None\n        detach: f32[1, 3, 3, 3] = torch.ops.aten.detach.default(relu)\n        sigmoid: f32[1, 3, 3, 3] = torch.ops.aten.sigmoid.default(relu);  relu = None\n        detach_1: f32[1, 3, 3, 3] = torch.ops.aten.detach.default(sigmoid)\n        relu_1: f32[1, 3, 3, 3] = torch.ops.aten.relu.default(sigmoid);  sigmoid = None\n        detach_2: f32[1, 3, 3, 3] = torch.ops.aten.detach.default(relu_1)\n        return relu_1\n        '

In [43]:
for n in exported.graph.nodes:
    print(n.meta.get("nn_module_stack", None))

None
None
None
None
None
None
None
None
None
None
None


In [44]:
print(type(exported))

<class 'torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl'>
